# Use Case A - Query builder (in progress)
***
* 2-grams, 3-grams, 4-grams
* Glossary articles only - SE articles to be added soon
* Many more matches expected with the addition of SE articles.


In [153]:


import re

import pandas as pd
import numpy as np

import pyodbc

import gensim

from datetime import datetime

import unicodedata

pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Load data from the database

In [154]:
c = pyodbc.connect('DSN=Virtuoso All;DBA=ESTAT;UID=user_name;PWD=password')
cursor = c.cursor()

In [155]:
for row in cursor.tables():
    print (row.table_name)

dat_3_grams
dat_4_grams
dat_4_grams_pickled
dat_article
dat_article_paragraph
dat_article_shared_link
dat_further_info
dat_glossary
dat_link_info
dat_link_tm
dat_paragraph_figure
dat_redirections
dat_related_concepts
dat_sources
dat_statistical_data
dat_tm_results
dat_tm_results_words
mod_article_division
mod_resource_information
mod_resource_type
mod_topic_models


### Definitions and last update from dat_glossary

In [156]:
GL_df = pd.read_sql("SELECT id,definition,last_update FROM dat_glossary", c) 
GL_df.head()

id  \
0   1   
1   2   
2   6   
3  10   
4  11   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

### Titles from dat_link_info (with resource_information_id=1)


In [157]:
titles = pd.read_sql("SELECT id,title,url FROM dat_link_info WHERE resource_information_id=1", c) 
#titles = titles[titles['url'].str.contains('title=Glossary:')]
titles.head()


,id,title,url
0,1,Accidents to persons caused by rolling stock in motion,https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Accidents_to_persons_caused_by_rolling_stock_in_motion
1,2,Railway,https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Railway
2,3,Railway safety statistics in the EU,https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Railway_safety_statistics_in_the_EU
3,4,Railway freight transport statistics,https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Railway_freight_transport_statistics
4,5,Railway passenger transport statistics - quarterly and annual data,https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Railway_passenger_transport_statistics_-_quarterly_and_annual_data


In [158]:
GL_df = GL_df.merge(titles,how='inner',on='id')
GL_df['orig definition'] = GL_df['definition'] ## original text for debugging
GL_df = GL_df[['id', 'title', 'definition', 'orig definition','url','last_update']]
   


In [159]:
GL_df = GL_df.replace('', np.nan) 
print(GL_df.isnull().sum())

## empty definitions

## print(GL_df[GL_df.isnull().any(axis=1)])

id                 0
title              0
definition         2
orig definition    2
url                0
last_update        0
dtype: int64


### Delete records with empty definitions, data cleansing

In [160]:
## drop records with any field empty
GL_df = GL_df.dropna(axis=0,how='any')
print(GL_df.isnull().sum())
GL_df.reset_index(inplace=True)

## MORE CASES CAUGHT + CORRECTIONS!
def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) ## NEW
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) ## NEW

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    x = x.encode('latin1').decode('utf-8') ## â\x80\x99
#This solution from here works well for me:

#soup = BeautifulSoup(r.content.decode('utf-8'),"lxml")
#If this doesn't work, adding .encode('latin1').decode('utf-8') after the .get_text() or .text also solves the issue.    
      
    return x


GL_df['title'] = GL_df['title'].apply(clean)
GL_df['definition'] = GL_df['definition'].apply(clean)
GL_df['url'] = GL_df['url'].apply(clean,quotes=False)
GL_df['url'] = GL_df['url'].apply(lambda x: [x])

GL_df.head()

id                 0
title              0
definition         0
orig definition    0
url                0
last_update        0
dtype: int64


index  id                                                   title  \
0      0   1  Accidents to persons caused by rolling stock in motion   
1      1   2                                                 Railway   
2      2   6                                        Accident at work   
3      3  10                                  Fatal accident at work   
4      4  11                              Non-fatal accident at work   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

### Delete "special" records

In [161]:
## Drop The records with definitions "The revision ..." and "Redirect to ..." 

idx = GL_df[GL_df['definition'].str.startswith('The revision #')].index
print(idx)
GL_df.drop(idx , inplace=True)
idx = GL_df[GL_df['definition'].str.startswith('Redirect to')].index
print(idx)
GL_df.drop(idx , inplace=True)
GL_df.reset_index(inplace=True)

Int64Index([ 170,  247,  280,  334,  382,  408,  411,  412,  413,  414,  490,
             514,  517,  578,  669,  705,  720,  754,  801,  861,  882,  930,
             931,  940,  977,  978, 1108, 1122, 1168, 1222],
           dtype='int64')
Int64Index([], dtype='int64')


### check for duplicate titles AND definitions

In [162]:
idx = GL_df[GL_df.duplicated(subset=['title','definition'], keep=False)].sort_values(by=['title','definition']).index
idx


Int64Index([1212, 1215], dtype='int64')

### Also produce processed file for debugging

In [163]:
## !!! below only for debugging
def file_name(pre,ext):
    current_time = datetime.now() 
    return pre + '_'+ str(current_time.month)+ '_' + str(current_time.day) + \
                 '_' + str(current_time.hour)+ '_' + str(current_time.minute)  +'.'+ext
    
outfile = file_name('D3_2_Glossary','xlsx')
GL_df.to_excel(outfile)


In [164]:
%%script false --no-raise-error
import ast

GL_df = pd.read_excel('D3_2_Glossary_6_8_23_9.xlsx',index_col=0)
GL_df['url'] = GL_df['url'].apply(lambda x: ast.literal_eval(x))
##GL_df

### Tokenize, remove stop-words and stem; keep also the original terms

In [165]:

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.porter import PorterStemmer

p = PorterStemmer()

def text_to_words(text,url):
    words = str(gensim.utils.simple_preprocess(text, deacc=True))
    words = remove_stopwords(words) 
    words = gensim.utils.tokenize(words)
        
    ## keep also original token!!! 
    words = [[p.stem(token),token,url] for token in words if len(p.stem(token)) >= 5] ##minimum length = 5 
    yield words        

texts=list()   

for i in range(len(GL_df)):
    texts.extend(text_to_words(GL_df.loc[i,'definition'],GL_df.loc[i,'url']))
    texts.extend(text_to_words(GL_df.loc[i,'title'],GL_df.loc[i,'url'])) 

In [166]:
#texts[0]

### Co-occurences: keys in n-grams are (n-1) tuples of stemmed tokens 

* Three dictionaries, for 2-,3-,and 4-grams
* For each key in a dictionary, the value is another (nested) dictionary with the **original terms**, their counts and the relevant URLs.
* Below all three dictionaries are constructed from the sequences of 4-grams.


In [167]:
##%%script false --no-raise-error
## Check also COLLOCATIONS: http://www.nltk.org/howto/collocations.html and http://www.nltk.org/api/nltk.html?highlight=ngram


from nltk import bigrams, trigrams, ngrams
#from collections import Counter, defaultdict


model2=dict()
model3=dict()
model4=dict()

def dict_insert(model,entered,proposed,new_urls_to_check):
    key_1 = model.get(entered)
    if key_1:
        key_2 = key_1.get(proposed)
        if key_2:
            key_2[0] +=1
            existing_urls = key_2[1]
            add_urls = [u for u in new_urls_to_check if u not in existing_urls]
            key_2.extend(add_urls)
        else:    
            key_1[proposed]= [1,new_urls_to_check]
    else:
        model[entered]={proposed:[1,new_urls_to_check]}
    return model            


# Co-occurences

for sentence in texts:
    pairs = [elem for elem in sentence] ## a list of 3-tuples (stemmed term, original term, list of URLs)
    if len(pairs) == 0: continue
    
    for first, second, third, fourth in ngrams(pairs,4): ## quadruplets of 3-tuples (stemmed term, original term, list of URLs)
        first_stem, first_orig, first_url = first
        second_stem, second_orig, second_url = second
        third_stem, third_orig, third_url = third
        fourth_stem, fourth_orig, fourth_url = fourth
       
        model2 = dict_insert(model2, first_stem, second_orig,list(set(first_url).intersection(second_url)))
        model2 = dict_insert(model2, second_stem, third_orig,list(set(second_url).intersection(third_url)))
        model2 = dict_insert(model2, third_stem, fourth_orig,list(set(third_url).intersection(fourth_url)))
        
        model3 = dict_insert(model3,(first_stem,second_stem),third_orig,list(set(first_url).intersection(*[second_url,third_url])))
        model3 = dict_insert(model3,(second_stem,third_stem),fourth_orig,list(set(second_url).intersection(*[third_url,fourth_url])))
        
        model4 = dict_insert(model4,(first_stem, second_stem, third_stem),fourth_orig,fourth_url)
   
        
## Transform counts to probabilities

for w1 in model2.keys():
    ssum = sum(model2[w1][w2][0] for w2 in model2[w1].keys())
    for w2 in model2[w1].keys():
        model2[w1][w2][0] /= ssum

for w1_w2 in model3.keys():
    ssum = sum(model3[w1_w2][w3][0] for w3 in model3[w1_w2].keys())
    for w3 in model3[w1_w2].keys():
        model3[w1_w2][w3][0] /= ssum

for w1_w2_w3 in model4.keys():
    ssum = sum(model4[w1_w2_w3][w4][0] for w4 in model4[w1_w2_w3].keys())
    for w4 in model4[w1_w2_w3].keys():
        model4[w1_w2_w3][w4][0] /= ssum


### Query Builder 1
***

In [168]:
import ipywidgets as widgets
layout = widgets.Layout(width='600px', height='30px')

In [169]:

def change_top_articles( Keywords):

    from operator import itemgetter
    p = PorterStemmer()
    
    last_match = ''
    
    #if not Keywords.endswith(' '):
    #    return None
    
    def test_and_back_step(x):
        mod_index = -1
        models = [model2,model3,model4]
        if len(x)==1: 
            x=x[0] ; model=models[0]
        elif len(x) ==2 or len(x) == 3:
            x=tuple(x) ; mod_index=len(x)-1; model=models[mod_index]
        elif len(x)==4:
            x=tuple(x[:3]) ; model=models[2]
        else:
            return None
        while not models[mod_index].get(x) and mod_index >=0:
            x=x[:-1]
            if len(x)==1 : x=x[0]
            mod_index -=1 ; model=models[mod_index] 
        return (model,x)    
            

    x = Keywords.split() 
    if len(x) ==0: 
        print()
        return
    x = [p.stem(el) for el in x]
    
    
    model,x = test_and_back_step(x)
    if not model.get(x):
        return None
    

    print()
    print('Based on last match: ',x,'\n')
    print('Suggestions, probabilities (in descending order) and relevant URLs: ')
    proposals = sorted([(k,v) for (k,v) in model[x].items()],key=itemgetter(1),reverse=True)
    last_match = x
    for key, value in proposals:
        print()
        print(key,': ',value[0])
        for url in value[1]:
            print(url)
   
    
def query_build1(value):
  style = {'description_width': 'initial'}
    
  Keywords = widgets.Text(
      value=value,
      placeholder='Type something',
      description='Keywords:',
      disabled=False
  )

  ui = widgets.HBox([Keywords])
  out = widgets.interactive_output(change_top_articles, {'Keywords': Keywords})
  display(ui, out)
    
query_build1(value='household')  

Output()